# Dataframe for Meta-Analysis

In [2]:
import pandas as pd
import datetime
import numpy as np
import patsy 

In [3]:
# Load arron-temp data
arron_temp = pd.read_csv("C:/Users/azdra/Documents/School/KU Leuven/Modern Data Analytics/mda_project/code/out/arron_temp_data.csv", sep = ",")

# Load mortality data
mort = pd.read_csv("C:/Users/azdra/Documents/School/KU Leuven/Modern Data Analytics/mda_project/code/out/mortality.csv", sep = ",")

In [4]:
mort

,YEAR,YEAR_WEEK,COD,ARRON,N_DEATHS,WEEK
0,2000,2000-001,natural,11000,31.0,1
1,2000,2000-001,natural,12000,10.0,1
2,2000,2000-001,natural,13000,14.0,1
3,2000,2000-001,natural,21000,43.0,1
4,2000,2000-001,natural,23000,14.0,1
...,...,...,...,...,...,...
45550,2019,2019-053,natural,84000,6.0,53
45551,2019,2019-053,natural,85000,5.0,53
45552,2019,2019-053,natural,91000,10.0,53
45553,2019,2019-053,natural,92000,19.0,53


In [5]:
arron_temp = arron_temp.drop(['Unnamed: 0', 'GRID_NO'], axis = 1)

In [6]:
arron_temp

,DAY,Nom_arrondissement,NIS_Code,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG,YEAR
0,20000101,Aalst,41000,8.4,4.7,6.6,2000.0
1,20000102,Aalst,41000,8.8,5.7,7.3,2000.0
2,20000103,Aalst,41000,9.5,7.1,8.3,2000.0
3,20000104,Aalst,41000,10.2,7.1,8.7,2000.0
4,20000105,Aalst,41000,7.2,2.1,4.7,2000.0
...,...,...,...,...,...,...,...
321415,20191227,Ypres,33000,8.0,6.9,7.4,2019.0
321416,20191228,Ypres,33000,7.5,5.7,6.6,2019.0
321417,20191229,Ypres,33000,4.8,0.5,2.7,2019.0
321418,20191230,Ypres,33000,7.7,1.2,4.5,2019.0


In [7]:
regions = arron_temp['Nom_arrondissement'].unique()
datalist = {}

for region in regions:
    datalist[region] = arron_temp[arron_temp['Nom_arrondissement'] == region]
    datalist[region]['tmean05'] = datalist[region]['TEMPERATURE_AVG'].rolling(window=6).mean()
    datalist[region]['WEEK'] = [int(datetime.date(x//10000, (x//100)%100, x%100).strftime("%V")) for x in datalist[region]['DAY']]
    datalist[region] = datalist[region][['DAY', 'YEAR', 'WEEK', 'Nom_arrondissement', 'NIS_Code', 'TEMPERATURE_MIN', 'TEMPERATURE_MAX', 'TEMPERATURE_AVG', 'tmean05']]
    datalist[region]['N_DEATHS'] = -1
    datalist[region].reset_index(drop = True)
    
    n_deaths=[]
    for i in range(0, len(datalist[region])):
        deaths = mort[(mort['YEAR']==datalist[region].iloc[i]['YEAR']) & 
                      (mort['WEEK']==datalist[region].iloc[i]['WEEK']) & 
                      (mort['ARRON']==datalist[region].iloc[i]['NIS_Code'])]['N_DEATHS']
        if len(deaths) ==0:
            n_deaths.append(np.NAN)
        else:
            n_deaths.append(deaths.iloc[0])
    datalist[region]['N_DEATHS'] = [round(d/7, 2) for d in n_deaths]

<ipython-input-7-9d9d995e7d2d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datalist[region]['tmean05'] = datalist[region]['TEMPERATURE_AVG'].rolling(window=6).mean()
<ipython-input-7-9d9d995e7d2d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datalist[region]['WEEK'] = [int(datetime.date(x//10000, (x//100)%100, x%100).strftime("%V")) for x in datalist[region]['DAY']]
<ipython-input-7-9d9d995e7d2d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [8]:
datalist

{'Aalst':            DAY    YEAR  WEEK Nom_arrondissement  NIS_Code  TEMPERATURE_MIN  \
 0     20000101  2000.0    52              Aalst     41000              4.7   
 1     20000102  2000.0    52              Aalst     41000              5.7   
 2     20000103  2000.0     1              Aalst     41000              7.1   
 3     20000104  2000.0     1              Aalst     41000              7.1   
 4     20000105  2000.0     1              Aalst     41000              2.1   
 ...        ...     ...   ...                ...       ...              ...   
 7300  20191227  2019.0    52              Aalst     41000              5.8   
 7301  20191228  2019.0    52              Aalst     41000              5.1   
 7302  20191229  2019.0    52              Aalst     41000             -0.9   
 7303  20191230  2019.0     1              Aalst     41000              2.0   
 7304  20191231  2019.0     1              Aalst     41000              0.5   
 
       TEMPERATURE_MAX  TEMPERATURE_AVG  

# Model

In [9]:
m = len(datalist)
m

44

In [10]:
# TEMPERATURE RANGES (FOR LAG 0-5)
min_tmean05 = pd.DataFrame([{k:(v['tmean05']).min() for k,v in datalist.items()}], index =["MIN"])
max_tmean05 = pd.DataFrame([{k:(v['tmean05']).max() for k,v in datalist.items()}], index = ["MAX"])
ranges = pd.concat([min_tmean05, max_tmean05], axis = 0).transpose()
ranges

,MIN,MAX
Aalst,-6.050000,25.483333
Antwerpen,-8.066667,25.366667
Arlon,-9.300000,29.366667
Ath,-6.800000,25.566667
Audenarde,-5.833333,25.633333
Bastogne,-9.900000,27.116667
Brugge,-5.016667,23.550000
Bruxelles-Capitale,-7.900000,25.350000
Charleroi,-9.033333,25.933333
Courtrai,-5.833333,25.633333


In [11]:
# DEFINE THE AVERAGE RANGE, CENTERING POINT, DEGREE AND TYPE OF THE SPLINE
# (THESE PARAMETERS CAN BE CHANGED BY THE USER FOR ADDITIONAL ANALYSES)
cen = 17
bounds = ranges.apply(np.mean)
degree = 2
type = "bs"
df = 6

# DEFINE THE KNOTS AT TEMPERATURE CORRESPONDING TO AVERAGE PERCENTILES
knotperc = [5, 35, 65, 95]
knots = pd.DataFrame(columns = ["5%", "35%", "65%", "95%"], index = regions)
for region in regions:
    perc = np.nanpercentile(datalist[region]["tmean05"], knotperc)
    knots.loc[region] = [perc[0], perc[1], perc[2], perc[3]]
knots
  

,5%,35%,65%,95%
Aalst,1.6325,8.41667,14.5167,19.9667
Antwerpen,1.5,8.35,14.4833,20.0167
Arlon,-0.816667,6.88333,13.8667,20.6175
Ath,1.41667,8.21667,14.3833,20.0342
Audenarde,1.78333,8.56667,14.6167,20.0167
Bastogne,-0.966667,6.455,13.2667,19.75
Brugge,2.35,8.38333,14.15,18.9333
Bruxelles-Capitale,1.31583,8.31667,14.5058,20.25
Charleroi,0.65,7.9,14.1667,19.9833
Courtrai,1.78333,8.56667,14.6167,20.0167


In [14]:
btmean05 = patsy.bs(x=datalist['Dinant']['tmean05'], knots=knots.loc['Dinant'], degree=degree, lower_bound=bounds[0], upper_bound=bounds[1])
btmean05
# import statsmodels.formula.api as smf
# import statsmodels.api as sm

# formula = 'death ~ btmean05 + dow + cubspl'
# model = smf.glm(formula = formula, data=rdata, family=sm.families.Poisson())
# result = model.fit()
# print(result.summary())

,0,1,2,3,4,5
87660,NaN,NaN,NaN,NaN,NaN,NaN
87661,NaN,NaN,NaN,NaN,NaN,NaN
87662,NaN,NaN,NaN,NaN,NaN,NaN
87663,NaN,NaN,NaN,NaN,NaN,NaN
87664,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
94960,0.021580,0.647700,0.330719,0.0,0.0,0.0
94961,0.039426,0.689178,0.271396,0.0,0.0,0.0
94962,0.067551,0.723890,0.208559,0.0,0.0,0.0
94963,0.081644,0.733783,0.184573,0.0,0.0,0.0


# Trying Things Out


In [ ]:
# #!git clone https://github.com/rpy2/rpy2.git
# #%cd C:\Users\azdra\Documents\School\KU Leuven\Modern Data Analytics\mda_project\code\rpy2
# #!python setup.py install


# import pandas as pd
# import rpy2.robjects as ro
# from rpy2.robjects.packages import importr
# from rpy2.robjects import pandas2ri

# from rpy2.robjects.conversion import localconverter

# r_df = ro.DataFrame({'int_values': ro.IntVector([1,2,3]),
#                      'str_values': ro.StrVector(['abc', 'def', 'ghi'])})

# r_df
# with localconverter(ro.default_converter + pandas2ri.converter):
#   pd_from_r_df = ro.conversion.rpy2py(r_df)

# pd_from_r_df

# df=datalist['Dinant']
# rdf = pandas2ri.py2ri_pandasdataframe(df)
# qpoisson = ro.r.glm('Claim_Count ~ Age + Vehicle_Use', data = rdf, family = ro.r('quasipoisson(link = &quot;log&quot;)'))
# #print ro.r.summary(qpoisson)

Cloning into 'rpy2'...


In [ ]:
# for region in regions:
#     # print iteration
#     print(region)

#     # load
#     data = datalist[region]

#     # Create the spline. Nodes and boundaries are fixed at same values
    



In [ ]:
# rdata = pd.read_csv("C:/Users/azdra/Documents/School/KU Leuven/Modern Data Analytics/mda_project/r_data.csv", sep = ",")
# rdata

# btmean05 = patsy.bs(x=rdata['tmean05'], knots=[2.850704,  7.800237, 12.898169, 18.153446], degree=degree, lower_bound=-2.523582, upper_bound=23.017419)

# import statsmodels.formula.api as smf
# import statsmodels.api as sm

# cubspl = patsy.cr(rdata['time'], 7*14)
# formula = 'death ~ btmean05 + dow + cubspl'
# model = smf.glm(formula = formula, data=rdata, family=sm.families.Poisson())
# result = model.fit()
# print(result.summary())